In [21]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

#BackgroundSubtractorMOG2
cap = cv2.VideoCapture('vedio_gray.avi')
#创建一个3*3的椭圆核
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
#创建BackgroundSubtractorMOG2
fgbg = cv2.createBackgroundSubtractorMOG2()

fps = 16  
size1 = (1792,1230) 
outpath1 = 'vedio_frame.avi'
videowriter1 = cv2.VideoWriter(outpath1,cv2.VideoWriter_fourcc('M','J','P','G'),fps,size1)

size2 = (1792,1230)
outpath2 = 'vedio_fgmask.avi'
videowriter2 = cv2.VideoWriter(outpath2,cv2.VideoWriter_fourcc('M','J','P','G'),fps,size2)

for i in range(114):
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)
    #形态学开运算去噪点
    # fgmask= cv2.fastNlMeansDenoisingColored(fgmask,None,10,10,7,21)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    #寻找视频中的轮廓
    contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for c in contours:
        #计算各轮廓的周长
        perimeter = cv2.arcLength(c,True)
        if perimeter > 188:
            #找到一个直矩形（不会旋转）
            x,y,w,h = cv2.boundingRect(c)
            #画出这个矩形
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)    
    videowriter1.write(frame)
    fgmask = cv2.merge((fgmask,fgmask,fgmask))
    videowriter2.write(fgmask)
    
# 关闭通道
cap.release()
videowriter1.release()
videowriter2.release()



In [9]:
import cv2
import numpy as np

camera = cv2.VideoCapture('vedio.avi') # 参数0表示第一个摄像头
# 判断视频是否打开
if (camera.isOpened()):
  print('Open')
else:
  print('摄像头未打开')
# 测试用,查看视频size
size = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
    int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('size:'+repr(size))

es = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 4))
kernel = np.ones((5, 5), np.uint8)
background = None

while True:
  # 读取视频流
  grabbed, frame_lwpCV = camera.read()
  # 对帧进行预处理，先转灰度图，再进行高斯滤波。
  # 用高斯滤波进行模糊处理，进行处理的原因：每个输入的视频都会因自然震动、光照变化或者摄像头本身等原因而产生噪声。对噪声进行平滑是为了避免在运动和跟踪时将其检测出来。
  gray_lwpCV = cv2.cvtColor(frame_lwpCV, cv2.COLOR_BGR2GRAY)
  gray_lwpCV = cv2.GaussianBlur(gray_lwpCV, (21, 21), 0)
 
  # 将第一帧设置为整个输入的背景
  if background is None:
    background = gray_lwpCV
    continue
  # 对于每个从背景之后读取的帧都会计算其与北京之间的差异，并得到一个差分图（different map）。
  # 还需要应用阈值来得到一幅黑白图像，并通过下面代码来膨胀（dilate）图像，从而对孔（hole）和缺陷（imperfection）进行归一化处理
  diff = cv2.absdiff(background, gray_lwpCV)
  diff = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)[1] # 二值化阈值处理
  diff = cv2.dilate(diff, es, iterations=2) # 形态学膨胀
 
  # 显示矩形框
  contours, hierarchy = cv2.findContours(diff.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # 该函数计算一幅图像中目标的轮廓
  for c in contours:
    if cv2.contourArea(c) < 1500: # 对于矩形区域，只显示大于给定阈值的轮廓，所以一些微小的变化不会显示。对于光照不变和噪声低的摄像头可不设定轮廓最小尺寸的阈值
      continue
    (x, y, w, h) = cv2.boundingRect(c) # 该函数计算矩形的边界框
    cv2.rectangle(frame_lwpCV, (x, y), (x+w, y+h), (0, 255, 0), 2)
 
  cv2.imshow('contours', frame_lwpCV)
  cv2.imshow('dis', diff)
 
  key = cv2.waitKey(1) & 0xFF
  # 按'q'健退出循环
  if key == ord('q'):
    break
# When everything done, release the capture
camera.release()
cv2.destroyAllWindows()

Open
size:(1792, 1230)


error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
